In [ ]:
import pandas as pd

resultados_exames = pd.read_csv('./exames.csv');
resultados_exames.head()

In [ ]:
from sklearn.model_selection import train_test_split
from numpy import random

random.seed(123143)

valores_exames = resultados_exames.drop(['id','diagnostico'], axis=1);
diagnosticos = resultados_exames.diagnostico

x_treino, x_teste, y_treino, y_teste = train_test_split(valores_exames, diagnosticos)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# classificador = RandomForestClassifier(n_estimators=100)
# classificador.fit(x_treino, y_treino)
# score = classificador.score(x_teste, y_teste)
# score

In [ ]:
resultados_exames.isnull().sum()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from numpy import random

random.seed(123143)

valores_exames = resultados_exames.drop(['id','diagnostico'], axis=1);
diagnosticos = resultados_exames.diagnostico
valores_exames_v1 = valores_exames.drop(columns='exame_33')

x_treino, x_teste, y_treino, y_teste = train_test_split(valores_exames_v1, diagnosticos, test_size=0.3)

classificador = RandomForestClassifier(n_estimators=100)
classificador.fit(x_treino, y_treino)
score = classificador.score(x_teste, y_teste)
score*100

In [ ]:
from sklearn.dummy import DummyClassifier
from numpy import random

random.seed(123143)
dummie_classifier = DummyClassifier(strategy='most_frequent')
dummie_classifier.fit(x_treino, y_treino)
dummie_score = dummie_classifier.score(x_teste, y_teste)
dummie_score*100

In [ ]:
dados_plot = pd.concat([diagnosticos, valores_exames_v1.iloc[:,0:10]], axis = 1)
dados_plot = pd.melt(dados_plot, id_vars='diagnostico', var_name='exames', value_name='valores')
dados_plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
sns.violinplot(x = 'exames', y = 'valores', hue = 'diagnostico', data = dados_plot)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaller = StandardScaler()
scaller.fit(valores_exames_v1)
valores_exames_v2 = scaller.transform(valores_exames_v1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

scaller = StandardScaler()
scaller.fit(valores_exames_v1)
valores_exames_v2 = pd.DataFrame(scaller.transform(valores_exames_v1), columns = valores_exames_v1.keys())

dados_plot = pd.concat([diagnosticos, valores_exames_v2.iloc[:,0:10]], axis = 1)
dados_plot = pd.melt(dados_plot, id_vars='diagnostico', var_name='exames', value_name='valores')
dados_plot

plt.figure(figsize=(10, 10))
plt.xticks(rotation=90)
sns.violinplot(x = 'exames', y = 'valores', hue = 'diagnostico', data = dados_plot, split = True)

In [ ]:
def grafico_violino(valores, inicio, fim):
    dados_plot = pd.concat([diagnosticos, valores.iloc[:,inicio:fim]], axis = 1)
    dados_plot = pd.melt(dados_plot, id_vars='diagnostico', var_name='exames', value_name='valores')
    dados_plot

    plt.figure(figsize=(10, 10))
    plt.xticks(rotation=90)
    sns.violinplot(x = 'exames', y = 'valores', hue = 'diagnostico', data = dados_plot, split = True)

grafico_violino(valores_exames_v2, 10, 21)

In [ ]:
grafico_violino(valores_exames_v2, 21, 32)

In [ ]:
valores_exames_v3 = valores_exames_v2.drop(columns = ['exame_29', 'exame_4'])

def classificar(valores):
    SEED = 1234
    random.seed(SEED)
    x_treino, x_teste, y_treino, y_teste = train_test_split(valores, diagnosticos, test_size=0.3)

    classificador = RandomForestClassifier(n_estimators=100)
    classificador.fit(x_treino, y_treino)
    score = classificador.score(x_teste, y_teste)
    print('Acurácia %.2f%%' % (score*100))

classificar(valores_exames_v3)

In [ ]:
correlation = valores_exames_v3.corr()

plt.figure(figsize=(20,20))
sns.heatmap(correlation, annot = True, fmt = '.1f')

In [ ]:
correlation_v1 = correlation[correlation > 0.99]
correlation_v2 = correlation_v1.sum()
correlation_v3 = correlation_v2[correlation_v2 > 1]
correlation_v3

In [ ]:
valores_exames_v4 = valores_exames_v3.drop(columns = correlation_v3.keys())
valores_exames_v4.head()

In [ ]:
classificar(valores_exames_v4)

In [ ]:
valores_exames_v5 = valores_exames_v3.drop(columns=['exame_3', 'exame_24'])
classificar(valores_exames_v5)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

k_best = SelectKBest(chi2, k = 5)

In [ ]:
valores_exames_v6 = valores_exames_v1.drop(columns=['exame_4', 'exame_29', 'exame_3', 'exame_24'])

In [ ]:
SEED = 1234
random.seed(SEED)
x_treino, x_teste, y_treino, y_teste = train_test_split(valores_exames_v6, diagnosticos, test_size=0.3)

k_best.fit(x_treino, y_treino)
treino_kbest = k_best.transform(x_treino)
teste_kbest = k_best.transform(x_teste)

In [ ]:
treino_kbest.shape

In [ ]:
classifier = RandomForestClassifier(n_estimators=100, random_state = 1234)
classifier.fit(treino_kbest, y_treino)
print('Acurácia %.2f%%' % (classifier.score(treino_kbest, y_treino)*100))

In [ ]:
from sklearn.metrics import confusion_matrix

matriz_confusao = confusion_matrix(y_teste, classifier.predict(teste_kbest))

In [ ]:
sns.set(font_scale=2)
sns.heatmap(matriz_confusao, annot = True, fmt='d').set(xlabel='Predição', ylabel = 'Real')

In [ ]:
from sklearn.feature_selection import RFE

SEED = 1234
random.seed(SEED)
x_treino, x_teste, y_treino, y_teste = train_test_split(valores_exames_v6, diagnosticos, test_size=0.3)

classifier = RandomForestClassifier(n_estimators=100, random_state = SEED)
classifier.fit(x_treino, y_treino)
selecionador_rfe = RFE(estimator=classifier, n_features_to_select = 5, step = 1)
selecionador_rfe.fit(x_treino, y_treino)
treino_rfe = selecionador_rfe.transform(x_treino)
teste_rfe = selecionador_rfe.transform(x_teste)

classifier.fit(treino_rfe, y_treino)
matriz_confusao = confusion_matrix(y_teste, classifier.predict(teste_rfe))
sns.set(font_scale=2)
sns.heatmap(matriz_confusao, annot = True, fmt='d').set(xlabel='Predição', ylabel = 'Real')
classifier.score(teste_rfe, y_teste)



In [ ]:
from sklearn.feature_selection import RFECV

SEED = 1234
random.seed(SEED)
x_treino, x_teste, y_treino, y_teste = train_test_split(valores_exames_v6, diagnosticos, test_size=0.3)

classifier = RandomForestClassifier(n_estimators=100, random_state = SEED)
classifier.fit(x_treino, y_treino)
selecionador_rfecv = RFECV(estimator=classifier, cv = 5, scoring='accuracy')
selecionador_rfecv.fit(x_treino, y_treino)
treino_rfecv = selecionador_rfecv.transform(x_treino)
teste_rfecv = selecionador_rfecv.transform(x_teste)

classifier.fit(treino_rfecv, y_treino)
matriz_confusao = confusion_matrix(y_teste, classifier.predict(teste_rfecv))
sns.set(font_scale=2)
sns.heatmap(matriz_confusao, annot = True, fmt='d').set(xlabel='Predição', ylabel = 'Real')
classifier.score(teste_rfecv, y_teste)

In [ ]:
x_treino.columns[selecionador_rfe.support_]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,8))
plt.xlabel('Número de exames')
plt.ylabel('Acurácia')

eixo_x = range(1, len(selecionador_rfecv.cv_results_['mean_test_score'])+1)
eixo_y = selecionador_rfecv.cv_results_['mean_test_score']
plt.plot(eixo_x, eixo_y)
plt.show()

In [ ]:
valores_exames_v7 = selecionador_rfe.transform(valores_exames_v6)

In [ ]:
valores_exames_v7.shape

In [ ]:
x = valores_exames_v7[:, 0]
y = valores_exames_v7[:, 1]
plt.figure(figsize=(14,8))
sns.scatterplot(x = x, y = y, hue = diagnosticos)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
valores_exames_v8 = pca.fit_transform(valores_exames_v5)

x = valores_exames_v8[:, 0]
y = valores_exames_v8[:, 1]
plt.figure(figsize=(14,8))
sns.scatterplot(x = x, y = y, hue = diagnosticos)

In [ ]:
valores_exames_v5

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components = 2)
valores_exames_v9 = tsne.fit_transform(valores_exames_v5)

x = valores_exames_v9[:, 0]
y = valores_exames_v9[:, 1]

plt.figure(figsize=(14,8))
sns.scatterplot(x = x, y = y, hue = diagnosticos)